# Classification
### Step 1: resize/crop images
Requires: `yolov5`, `beescrape.py`

In [ ]:
SRC_DIR = '/home/wilber/Documents/RESEARCH/research/beespotter'
OUT_DIR = '/m2docs/res/data'

# Species with >= 1000 images:
labels = ['Apis_mellifera','Bombus_impatiens','Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']

